In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("derek-thomas/ScienceQA")
validation_data = dataset["validation"]
filtered_validation_data = validation_data.filter(lambda example: example["lecture"] != "" and example["solution"] != "")
filtered_validation_data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

(…)-00000-of-00001-1028f23e353fbe3e.parquet:   0%|          | 0.00/377M [00:00<?, ?B/s]

(…)-00000-of-00001-6c7328ff6c84284c.parquet:   0%|          | 0.00/126M [00:00<?, ?B/s]

(…)-00000-of-00001-f0e719df791966ff.parquet:   0%|          | 0.00/122M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12726 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4241 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4241 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'question', 'choices', 'answer', 'hint', 'task', 'grade', 'subject', 'topic', 'category', 'skill', 'lecture', 'solution'],
    num_rows: 3216
})

In [ ]:
import pandas as pd
import numpy as np
import csv
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#this notebook is restoring the script used for postprocessing + evaluation (benchmarking.ipynb)

In [ ]:
project_dir = "foundation_models"
input_folder = "benchmarking_val/outputs/llava_1_5_7B/json_outputs"
output_folder = "benchmarking_val/outputs/llava_1_5_7B/parsed_json"
full_input_dir = os.path.join("/content/drive/MyDrive", project_dir, input_folder)
full_res_dir = os.path.join("/content/drive/MyDrive", project_dir, output_folder)
files = os.listdir(full_input_dir)
files = [x for x in files if x.startswith("llava")]
files

['llava_1_5_7B_val_output_setting_1.csv',
 'llava_1_5_7B_val_output_setting_3.csv',
 'llava_1_5_7B_val_output_setting_4.csv',
 'llava_1_5_7B_val_output_setting_2.csv']

In [ ]:
file_path = os.path.join(full_input_dir, files[0])
df = pd.read_csv(file_path, delimiter="\t")
print(f"Loaded dataframe: {file_path} with {len(df)} rows.")

Loaded dataframe: /content/drive/MyDrive/foundation_models/benchmarking_val/outputs/llava_1_5_7B/json_outputs/llava_1_5_7B_val_output_setting_1.csv with 3216 rows.


In [ ]:
df

,idx,input,output,subject
0,0,Question: What does the verbal irony in this t...,"{\n""answer"": ""The snoring is loud."",\n""soluti...",language science
1,1,Question: Which animal's mouth is also adapted...,"{\n""answer"": ""sturgeon"",\n""solution"": ""Sturge...",natural science
2,2,Question: Is this a sentence fragment?\nDuring...,No,language science
3,3,Question: Which correctly shows the title of a...,"{\n""answer"": ""A Breath of Fresh Air"",\n""solut...",language science
4,4,Question: Does this passage describe the weath...,"{\n""answer"": ""climate"",\n""solution"": ""The ima...",natural science
...,...,...,...,...
3211,3211,Question: How long is an adult great white sha...,"{\n""answer"": ""6 meters"",\n""solution"": ""A grea...",natural science
3212,3212,Question: Which animal is also adapted to be c...,"{\n""answer"": ""Namaqua chameleon"",\n""solution""...",natural science
3213,3213,Question: Compare the motion of two fish. Whic...,"{\n""answer"": ""a fish that moved 25kilometers ...",natural science
3214,3214,Question: Which continent is highlighted?\n Ta...,"{\n""answer"": ""Europe"",\n""solution"": ""The cont...",social science


In [ ]:
import re
import json
# Dasha's function that I used for consistency when I did evaluation of LLaVA
def parse_json(text):
    # remove additional comments from LLM
    cleaned_text = re.sub("```|\n|json", "", str(text)).strip()
    # terminate string if it was terminated early and JSON is incomplete
    if cleaned_text[-2:] != '"}':
        cleaned_text += '"}'
    empty_output = "", ""
    try:
        parsed_output = json.loads(cleaned_text)
    except Exception as e:
        # print(e)
        # print(text)
        return cleaned_text, "" #return full text as answer and "" as solution

    if type(parsed_output) == dict:
        if "answer" in parsed_output.keys():
            answer = parsed_output["answer"]
        else:
            answer = ""
        if "solution" in parsed_output.keys():
            solution = parsed_output["solution"]
        else:
            solution = ""
        return answer, solution
    else:
        return empty_output

In [ ]:
# This cell also repeats the steps I did while evaluating LLaVA outputs with benchmarking.ipynb. Commented out are the original lines, below are the lines in a new environment
# I am also not doing evaluation now -- only producing answer and solution columns as agreed for consistency
# The script is merely for the sake of documentation

# setting_map = ['QTCH', 'QTCHL', 'QTCHLS', 'QTCHS']
setting_map = ['1', '2', '3', '4']
# device = 'mps'

# def eval(RES_DIR:str, MODEL_NAME:str): #I kept this horrible name initially trying to rewrite as little given code as possible but I can't that's just too bad.
# I'm not sure why ScienceQA creators decided in favor of so many bad names -- maybe they're trying to create noisy data knowing that code generation models will be trained on open github repositories?
# if not I think they used this whale for their work: https://www.youtube.com/watch?v=q6dKllQzVxU&t=8s
def postprocess_and_evaluate(RES_DIR:str, MODEL_NAME:str):
    for i in range(4):
        # load data
        SETTING = setting_map[i]
        SETTING_NUM = i+1
        print(f"Metrics for setting {SETTING_NUM} {SETTING}")
        # path_to_df = f"benchmarking/{MODEL_NAME}/{MODEL_NAME}_val_output_setting_{SETTING}.csv"
        path_to_df = f"{full_input_dir}/{MODEL_NAME}_val_output_setting_{SETTING}.csv"
        df = pd.read_csv(path_to_df, sep="\t", header=0)[["idx", "input", "output"]] #avoid duplicate columns
        print(f"Loaded dataframe {path_to_df} with {len(df)} rows")
        # append validation data as accuracy evaluation will depend on it (as well as comparing selected answer to the given choices)
        hf_df = filtered_validation_data.to_pandas()
        hf_df = hf_df.reset_index()
        hf_df = hf_df.rename(columns={"index": "idx"})
        # check correct number of processed datapoints
        if len(hf_df) != len(df):
            print(f"WARNING: Data may be incomplete. Original dataset has {len(hf_df)} rows")
        df = pd.merge(df, hf_df, on="idx", how="inner")
        print(f"Merged dataframe with metadata has {len(df)} rows")
        # parse generated answer json
        # answer_pred_col, solution_pred_col = "answer_output", "solution_output"
        answer_pred_col, solution_pred_col = "answer_str", "explanation"
        df[[answer_pred_col, solution_pred_col]] = df["output"].apply(parse_json).apply(pd.Series)
        # extract answer number (index in choices)
        # df[answer_pred_col] = df.apply(lambda row: find_answer_num(row[answer_pred_col], row["choices"]), axis=1)
        df['answer'] = df.apply(lambda row: find_answer_num(row[answer_pred_col], row["choices"]), axis=1)
        # commenting out evaluation to keep only postprocessing
        # solution_preds =  df[solution_pred_col].tolist()
        # #accuracy per subject calculations to evaluate answers
        # scores = get_scores(df)
        # print_scores(scores)
        # #textual similarity to evaluate solutions
        # metrics = calculate_metrics_solutions(solution_preds, df)
        # scores.update(metrics)
        # dict_save = {'model': MODEL_NAME, 'setting': SETTING}
        # dict_save.update(scores)
        # df_save = pd.DataFrame(dict_save)
        # df_save.to_csv(f"{RES_DIR}/{MODEL_NAME}_val_metrics_setting_{SETTING_NUM}.csv", sep="\t", encoding="utf-8")
        df[["idx", "input", "output", "subject", "answer", "explanation", "answer_str"]].to_csv(f"{RES_DIR}/{MODEL_NAME}_val_output_setting_{SETTING_NUM}.csv", sep="\t", encoding="utf-8")

def find_answer_num(answer_output, choices):
    try:
        # ignore any uncoventional format of answer output; if it is a list or anything else it is simply incorrect, pass
        return int(np.where(choices == str(answer_output))[0][0])
    except IndexError:
        return -1

In [ ]:
postprocess_and_evaluate(full_res_dir, "llava_1_5_7B")

Metrics for setting 1 1
Loaded dataframe /content/drive/MyDrive/foundation_models/benchmarking_val/outputs/llava_1_5_7B/json_outputs/llava_1_5_7B_val_output_setting_1.csv with 3216 rows
Merged dataframe with metadata has 3216 rows
Metrics for setting 2 2
Loaded dataframe /content/drive/MyDrive/foundation_models/benchmarking_val/outputs/llava_1_5_7B/json_outputs/llava_1_5_7B_val_output_setting_2.csv with 3216 rows
Merged dataframe with metadata has 3216 rows
Metrics for setting 3 3
Loaded dataframe /content/drive/MyDrive/foundation_models/benchmarking_val/outputs/llava_1_5_7B/json_outputs/llava_1_5_7B_val_output_setting_3.csv with 3216 rows
Merged dataframe with metadata has 3216 rows
Metrics for setting 4 4
Loaded dataframe /content/drive/MyDrive/foundation_models/benchmarking_val/outputs/llava_1_5_7B/json_outputs/llava_1_5_7B_val_output_setting_4.csv with 3216 rows
Merged dataframe with metadata has 3216 rows


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/foundation_models/benchmarking_val/outputs/llava_1_5_7B/parsed_json/llava_1_5_7B_val_output_setting_1.csv", delimiter="\t")
df.value_counts("answer") #roughly the same amount of -1 as pixtral

,count
answer,
1,1269
0,1082
-1,508
2,294
3,61
4,2
